In [ ]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the "../input/" directory.
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('./dataset'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [1]:
import os
# import jieba
import swifter
import numpy as np
import pandas as pd

from tqdm import tqdm
# from pypinyin import lazy_pinyin, Style, load_single_dict, load_phrases_dict

D:\Softwares\Anaconda3\envs\test\lib\site-packages\tqdm\autonotebook\__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
style = Style.TONE3

In [3]:
with open("./user_dict.txt", "w", encoding="utf8") as fw:
    fw.write("""[*] 4
[LAUGH] 4
[SONANT] 4
[ENs] 4
[MUSIC] 4
君意 4
+ 10""")
jieba.load_userdict("user_dict.txt")

# import shutil
# shutil.copytree("../input/dpcode0/DeepSpeechRecognition-master/", "./DeepSpeechRecognition-master/")

# os.listdir("./DeepSpeechRecognition-master/")

dev = pd.read_csv("./dataset/dev.csv")
test = pd.read_csv("./dataset/test.csv")
train = pd.read_csv("./dataset/train.csv")

# train.words.value_counts()[:20]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lizeda\AppData\Local\Temp\jieba.cache
Loading model cost 1.019 seconds.
Prefix dict has been built successfully.


In [ ]:
len([i for i in dev.words if "+" in i])

In [4]:
def lcut(x):
    if x.startswith("["):
        return [x]
    return jieba.lcut(x)

In [5]:
en2cn = {'A': '诶','B': '必','C': '细','D': '地','E': '亿','F': 'f','G': '计',
'H': 'h','I': '爱','J': '戒','K': '尅','L': 'l','M': 'm','N': '恩','O': '欧','P': '批','Q': 'q',
'R': '儿','S': 's','T': '剃','U': '优','V': '微','W': 'z','X': 'x','Y': '外','Z': 'z'}

en2cn_dict = {}
for i in en2cn:
    en2cn_dict[i] = lazy_pinyin(en2cn[i], style=style)[0]
    en2cn_dict[i.lower()] = en2cn_dict[i]
# load_single_dict(en2cn_dict)

In [6]:
def errorpy(words):
    words = words.strip()
    if words.isalpha() and len(words)==1:
        return en2cn_dict[words]
    elif len(words)>1:
        print(words)
        return [en2cn_dict[i]  if i.isalpha() and len(i)==1 else i for i in words.split(" ")]                  
    else:
        return words

In [7]:
print(lazy_pinyin('12个[UNK]的 小app兔子 r n g im a teacher ', style=style, errors=errorpy))

12
[UNK]
app
r n g im a teacher
['12', 'ge4', '[UNK]', 'de', 'xiao3', 'app', 'tu4', 'zi', 'er2', 'en1', 'ji4', 'im', 'ei2', 'teacher']


In [9]:
import string, re
# punctuation = string.punctuation + """！？｡。＂＃＄％＆＇（）＊＋，－／：；＝＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘'‛“”„‟…‧﹏."""
punctuation = '!"#$%&\'()*+,-./:;=?@\\^_`{|}~' + """！？｡。＂＃＄％＆＇（）＊＋，－／：；＝＠＼＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘'‛“”„‟…‧﹏."""

In [23]:
read_files = ["dev.txt"]#["dev.csv"]#   #["train.csv"] #
wav_lst = []
pny_lst = []
han_lst = []
for file in read_files:
    if "csv" in file:
        data = pd.read_csv("./dataset/"+file)
        data = data.loc[~data.words.isin(['+', '[SONANT]', '[ENS]', '[*]', '[LAUGH]', '[MUSIC]'])]
        data.words = data.words.apply(lambda x:re.sub("[{}]+".format(punctuation), " ", x).strip() )
        data["wav_file"] = data["Unnamed: 0"].swifter.apply(lambda x: "./dataset/train_bytes/wave_"+str(x)+".csv")
        data["pny"] = data["words"].swifter.apply(lambda x: lazy_pinyin(x, style=style, errors=errorpy))
    else:
        print('load ', file, ' data...')
        sub_file = './' + file
        with open(sub_file, 'r', encoding='utf8') as f:
            data = f.readlines()
        for line in tqdm(data):
            wav_file, pny, han = line.split('\t')
            wav_lst.append(wav_file)
            pny_lst.append(pny.split(' '))
            han_lst.append(han.strip('\n'))

load  dev.txt  data...


100%|██████████████████████████████████████████████████████████████████████████| 6086/6086 [00:00<00:00, 338997.80it/s]


In [30]:
def add_pinyin(data):
    data = data.loc[~data.words.isin(['+', '[SONANT]', '[ENS]', '[*]', '[LAUGH]', '[MUSIC]'])]
    data.loc[:, "words"] = data.words.swifter.apply(lambda x:re.sub("[{}]+".format(punctuation), " ", x).strip() )
    data["wav_file"] = data["Unnamed: 0"].swifter.apply(lambda x: "./dataset/train_bytes/wave_"+str(x)+".bin")
    data["pny"] = data["words"].swifter.apply(lambda x: lazy_pinyin(x, style=style, errors=errorpy))
    return data

In [31]:
train_df = add_pinyin(train)
dev_df = add_pinyin(dev)

D:\Program Files\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


D:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


app
olay
olay
p k
m a c
b e t
a j
a d
a d
a d
z k
mvp
mvp
mvp
m v p
low
m v p
pk
nba
nba
logo
nba
ok
nba
nba
nba
nba
nba
cba
mvp
mvp
mvp
mvp
nba
nba
nba
cba
mvp
cba
nba
nba
nba
cba
nba
cba
nba
nba
cba
cba
mvp
nba
cba
A A
D N A
R N A
qq
vip
n b a
n b a
n b a
nba
nba
nba
nba
nba
nba
nba
vip
vip
buff
buff
buff
buff
solo
solo
solo
adc
adc
q q
q q
q q
o k
pk
pk
man
out
see sky
papi
papi
q q
q q
y s l
y s l
iphone x iphone x s
iphone
iPhone
cctv
running
gai
rapper
freestyle
flow
popping
old school
pk
pk
popping
tv
buff
buff
buff
buff
buff
buff
double kill
biu
biu
w s a d
w s a d
oppo
oppo
buff
boss
man
exo
exo
exo
exo
m m
exo
exo
exo
exo
tfboys
nine
snh
b b b b
miniso
b b
b b
c c
b b
b b
b b
b b
kiko
t f
t f
a j
a j
a j
t f
clean
c l e a n
bb
c c
cc
w i s
v i s
m a c
tfboys
cosplay
cosplay
s n h four eight
s n h four eight
cosplay
mv
mv
boss
boss
hi
app
exo
mike
man
t f boys
t f boys
t f boys
video
angelababy
a b
angelababy
iPhone x
broke
a c c a
a c c a
exo
exo exo
exo
exo
kbs
part
kbs
mons

D:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


fish
dolphin
monkey
panda
rabbit
elephant
tiger
lion
alligator
Giraffe
Giraffe
nfc
a i
ai
ai
ai
pro
z z
bose
jbl
bose
u e
m p
m p
m p
m p
hifi
hifi
hifi
m p
c d
c d
m p
c d
hifi
hifi
c d
c d
c d
c d
m p
a p e
f l a c
jbl
jbl
jbl
hdr
tcl
iphone
oppo
vivo
oppo find
vivo find
vivo
nex nex
vivo
vivo
vivo
google
google
lg
tcl
tcl htc
google pixel
google pixel
google
nexus
ipad
c c
c c
java
d h a
d h a
d n a
v c
v c
v c
v c
v c
hello
hello
ok
hello
hello
OK
OK
OK
ok
ok
m v p
i t
i t
XFun
ok
ok
r n g
p p k
c s
c s c f
c f
qq
[UNK]
c p
c p
[UNK]
d i y
over
ip



In [32]:
train_df.pny = train_df.pny.swifter.apply(lambda x:" ".join(x))
dev_df.pny = dev_df.pny.swifter.apply(lambda x:" ".join(x))

D:\Program Files\Anaconda3\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [41]:
dev_df.wav_file = dev_df.apply(lambda x: "./dataset/dev(SPK)/dev_byte(%s)/wave_"%(x["speaker"])+str(x["Unnamed: 0"])+".bin", axis=1)

D:\Program Files\Anaconda3\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [33]:
train_df.loc[:, ["wav_file", "pny", "words"]].to_csv("train.txt", index=False, header=False, sep="\t")

In [45]:
dev_df.loc[~dev_df.speaker.isna(), ["wav_file", "pny", "words"]].to_csv("dev.txt", index=False, header=False, sep="\t")

In [2]:
from utils import compute_fbank, multi_task, generate_fbank

import time
import multiprocessing as mlp

In [3]:
train_df = pd.read_csv("./dataset/dev.txt", delimiter="\t", header=None)

In [4]:
train_df.columns = ["wav_file", "pny", "words"]
train_df.head()

,wav_file,pny,words
0,./dataset/dev(SPK)/dev_byte(SPK001)/wave_1.bin,niu1 niu1 jin1 tian1 wo3 men liao2 shen2 me ne,妞妞 今天我们聊什么呢
1,./dataset/dev(SPK)/dev_byte(SPK002)/wave_2.bin,wo3 men jin1 tian1 liao2 dong4 wu4,我们今天聊动物
2,./dataset/dev(SPK)/dev_byte(SPK001)/wave_3.bin,hao3 ba,好吧
3,./dataset/dev(SPK)/dev_byte(SPK001)/wave_5.bin,na4 ni3 shuo1 yi1 shuo1 dong4 wu4 ba,那你说一说动物吧
4,./dataset/dev(SPK)/dev_byte(SPK002)/wave_6.bin,a1 yi2 ni3 xi3 huan1 shen2 me,阿姨你喜欢什么


In [9]:
for i in os.listdir("./dataset/train_bytes/"):
    if "npz" not in i and "bin" not in i:
        _path = os.path.join("./dataset/train_bytes/", i)
        os.rename(_path, _path + ".bin")

In [8]:
np.load("./dataset/train_bytes/wave_873.npz")["data"]

array([[3.24859839, 6.06366269, 6.34028129, ..., 0.74907302, 1.87378113,
        2.41111297],
       [5.4403597 , 6.27562947, 6.87843905, ..., 1.78649284, 2.7536022 ,
        0.76069381],
       [4.72121008, 5.54630119, 6.02107866, ..., 1.87753565, 0.9248445 ,
        1.65966227],
       ...,
       [3.46930624, 4.14616927, 5.06861765, ..., 1.28698306, 2.21705445,
        0.90436631],
       [0.3783282 , 4.66558223, 5.3631227 , ..., 1.74361356, 1.98506984,
        1.60256936],
       [3.42243883, 4.17866339, 4.75218618, ..., 2.01441578, 2.030116  ,
        2.20481057]])

In [10]:
# def generate_fbank(x):
#     data = compute_fbank(x)
#     path = x.replace(".csv", "")
#     if not os.path.exists(os.path.dirname(path)):
#         print("making dir:", os.path.dirname(path))
#         os.makedirs(os.path.dirname(path))
#     np.savez_compressed(path, data=data)
#     return path

# def multi_task(data, s_type):
#     data["fbank_path"] =  data['wav_file'].swifter.apply(lambda x: generate_fbank(x))
#     return data

train_files = []

train_files = train_df.loc[~train_df.wav_file.swifter.apply(lambda x: os.path.exists(x.replace("bin", "npz")))]
# train
t0 = time.time()
N = train_files.shape[0]
print(N)
num_cpu = 20 # cpu数量
pool = mlp.Pool(num_cpu)

results = []
batch_num = N // num_cpu
print('batch_num',batch_num)
for i in range(num_cpu):
    if i == num_cpu-1:
        offset = N
    else:
        offset = (i+1)*batch_num
    task_train = train_files.iloc[i*batch_num : offset]
    print('task',i,'shape',len(task_train))
    result = pool.apply_async(multi_task,(task_train, 'train',))
    results.append(result)
pool.close()
pool.join()
print('finished',len(results))

t1 = time.time()
print('time', int(t1 - t0), 's')
trs = pd.DataFrame()
for result in tqdm(results):
    trs = pd.concat([trs, result.get()])
print(trs.shape[0])


6085
batch_num 304
task 0 shape 304
task 1 shape 304
task 2 shape 304
task 3 shape 304
task 4 shape 304
task 5 shape 304
task 6 shape 304
task 7 shape 304
task 8 shape 304
task 9 shape 304
task 10 shape 304
task 11 shape 304
task 12 shape 304
task 13 shape 304
task 14 shape 304
task 15 shape 304
task 16 shape 304
task 17 shape 304
task 18 shape 304
task 19 shape 309
finished 20
time 930 s


100%|█████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 181.81it/s]


6085


In [ ]:
def mk_am_vocab(data):
    vocab = []
    for line in tqdm(data):
        for pny in line:
            if pny not in vocab:
                vocab.append(pny)
    vocab.append('_')
    return vocab

def mk_lm_pny_vocab(data):
    vocab = ['<PAD>']
    for line in tqdm(data):
        for pny in line:
            if pny not in vocab:
                vocab.append(pny)
    return vocab

In [ ]:
len({j for p in pny_lst for j in p})

In [ ]:
len(mk_am_vocab(pny_lst))

In [ ]:
print('make am vocab...')
am_vocab_tr = mk_am_vocab(train_df.pny)
print('make lm pinyin vocab...')
pny_vocab_tr = mk_lm_pny_vocab(train_df.pny)
print('make am vocab...')
am_vocab_dev = mk_am_vocab(dev_df.pny)
print('make lm pinyin vocab...')
pny_vocab_dev = mk_lm_pny_vocab(dev_df.pny)

In [ ]:
count = 0
for i in set(pny_vocab_tr):
    if i.isalpha():
        print(i)
        count+=1
count

In [ ]:
count = 0
for i in set(pny_vocab_dev):
    if i.isalpha():
        print(i)
        count+=1
count

In [ ]:
alpha_list_tr = []
for i in set(pny_vocab_tr):
    if i.isalpha():
        alpha_list_tr.append(i)
alpha_list_tr

In [ ]:
dev_alpha_list = []
for i in set(pny_vocab_dev):
    if i.isalpha():
        dev_alpha_list.append(i)

In [ ]:
print("全是字母的：", alpha_list_tr)

In [ ]:
print("全是字母的：", dev_alpha_list)

In [ ]:
# 本地中文拼音库
with open("pinyin.txt", "r") as f:
    pinyinLib = f.read().split("\n")

en_list = []
for i in alpha_list_tr:
    if i not in pinyinLib and len(i)>0:
        en_list.append(i)

In [ ]:
en_list

In [ ]:
dev_en_list = []
for i in dev_alpha_list:
    if i not in pinyinLib and len(i)>0:
        dev_en_list.append(i)

In [ ]:
print("英文字母：", en_list)

In [ ]:
print("英文字母：", dev_en_list)

In [ ]:
import json

with open("./pny_dict.json", "w") as fw:
    fw.write(json.dumps({"train": list(train_pny), "dev":list(dev_pny)}))

In [ ]:
def is_Chinese(word):
    for ch in word:
        if '\u4e00' <= ch <= '\u9fff':
            return True
    return False

In [ ]:
vocab = []
line = "12个[UNK]的 小app兔子mike r n g im a   teacher ".split(' ')
for patch in line:
    if patch.encode("utf-8").isalpha():
        if patch not in vocab:
            vocab.append(patch)
    elif patch == "[UNK]":
        print(patch)
    else:
        en_str = ""
        for han in patch:
            if not is_Chinese(han):
                en_str += han
            elif is_Chinese(han):            
                if en_str != "" and en_str not in vocab:
                    vocab.append(en_str)
                    en_str = ""
                if han not in vocab:
                    vocab.append(han)
            else:
                print("ERROR:", han)
        if en_str != "" and en_str not in vocab:
            vocab.append(en_str) 
            en_str = ""
vocab

In [ ]:
def mk_lm_han_vocab(data):
    vocab = ['<PAD>']
    for line in tqdm(data):
        if line in vocab:
            continue
        line = line.split(' ')
        for patch in line:
            if patch.encode("utf-8").isalpha():
                if patch not in vocab:
                    vocab.append(patch)
            elif patch == "[UNK]":
                print(patch)
            else:
                en_str = ""
                for han in patch:
                    if not is_Chinese(han):
                        en_str += han
                    elif is_Chinese(han):            
                        if en_str != "" and en_str not in vocab:
                            vocab.append(en_str)
                            en_str = ""
                        if han not in vocab:
                            vocab.append(han)
                    else:
                        print("ERROR:", han)
                if en_str != "" and en_str not in vocab:
                    vocab.append(en_str) 
                    en_str = ""
    return vocab

In [ ]:
print('make lm hanzi vocab...')
han_vocab = mk_lm_han_vocab(train_df.words)

In [ ]:
set(dev_pny_vocab).difference(set(pny_vocab))

In [ ]:
len(set(han_vocab)),len(set(han_vocab))